# Pruebas de modelo para analizar sentimiento de noticias en inglés

---


### Paso 1: Configuración del Entorno y Librerías

Se instalan las bibliotecas necesarias y se importan los módulos clave. `transformers` es el módulo principal para trabajar con modelos de Hugging Face. `pandas` se utiliza para la manipulación de datos, `torch` para el cómputo con la GPU, y `re` e `io` para manejar la lectura y limpieza del archivo de manera eficiente.

In [ ]:

import pandas as pd
import re
import io
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

Mounted at /content/drive


### Paso 2: Carga y Preparación de los Datos

En este bloque, montamos Google Drive para acceder a nuestros archivos. El código se centra en una técnica de procesamiento en memoria (`io.StringIO`) para leer y limpiar el archivo CSV sin tener que crear y guardar un nuevo archivo temporal. Esto es crucial para manejar grandes volúmenes de datos de forma rápida y eficiente en un entorno como Colab.

In [ ]:
# Definir las rutas de los archivos
# Ruta del archivo original en tu Google Drive (solo necesitamos este)
original_file_path = 'NOTICIAS_POR_ANALIZAR.csv'

In [ ]:
# Leer y limpiar el contenido del archivo en memoria (sin crear un archivo limpio)
print(f"Leyendo y limpiando el contenido de '{original_file_path}' en memoria...")
cleaned_lines = []
try:
    with open(original_file_path, 'r', encoding='utf-8') as infile:
        header = next(infile)
        cleaned_lines.append(header) # Guardar el encabezado

        for line in infile:
            if line.strip().startswith('"') and ',' in line[1:]:
                # Reemplazar el formato incorrecto en memoria
                line_fixed = re.sub(r'^"(\d+),""', r'\1,"', line.strip())
                line_fixed = re.sub(r'""', r'"', line_fixed)
                cleaned_lines.append(line_fixed + '\n')
            else:
                cleaned_lines.append(line)

    # Unir las líneas limpias en una sola cadena para que Pandas pueda leerlo
    cleaned_csv_string = "".join(cleaned_lines)

    # Usar `io.StringIO` para tratar la cadena como un archivo en memoria
    # Esto evita tener que guardar un archivo limpio en el disco.
    cleaned_csv_file = io.StringIO(cleaned_csv_string)

    # Leer el archivo virtual en un DataFrame de Pandas
    df = pd.read_csv(cleaned_csv_file)
    print(f"DataFrame cargado y limpio en memoria con {len(df)} registros.")
    print(df.head())

except FileNotFoundError:
    print(f"Error: El archivo original '{original_file_path}' no se encontró. Verifica la ruta.")
    df = None

Leyendo y limpiando el contenido de '/content/drive/MyDrive/UCM/TFM/NOTICIAS_POR_ANALIZAR.csv' en memoria...


/tmp/ipython-input-3473739433.py:26: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(cleaned_csv_file)


DataFrame cargado y limpio en memoria con 1069449 registros.
                                     ID       FECHA  \
0  70ab9305-75e8-4cb9-9ef6-c5286c3d07f8  2024-07-08   
1  ce5fcf5b-1b24-49eb-a162-2195c738abd6  2025-08-02   
2  9294d502-a662-4674-baaa-2914dbc4f637  2025-07-29   
3  9b448a0f-aa4e-4245-bae4-aaf725bf2e92  2025-07-22   
4  d666f36b-b0c0-45fa-a13d-b06d284d763c  2025-07-24   

                                             TITULAR  \
0  Trabajadores de aeropuertos franceses convocan...   
1  How South Korea's K-beauty industry is being h...   
2  Carrefour y Alcampo, cara y cruz: la primera a...   
3  Sánchez Llibre, presidente de Foment, agradece...   
4   'Ringrose doesn't realise magnitude of his call'   

                                         URL_ARCHIVO   FUENTE IDIOMA  \
0  https://web.archive.org/web/20240709000354/htt...      ABC     es   
1  https://web.archive.org/web/20250803061923/htt...      BBC     en   
2  https://web.archive.org/web/20250727135337/htt...  E

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1069449 entries, 0 to 1069448
Data columns (total 10 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   ID                     1069449 non-null  object 
 1   FECHA                  1069449 non-null  object 
 2   TITULAR                1069449 non-null  object 
 3   URL_ARCHIVO            1069449 non-null  object 
 4   FUENTE                 1069449 non-null  object 
 5   IDIOMA                 1069449 non-null  object 
 6   SENTIMIENTO_RESULTADO  30 non-null       object 
 7   PROBABILIDAD_POSITIVO  30 non-null       float64
 8   PROBABILIDAD_NEGATIVA  30 non-null       float64
 9   PROBABILIDAD_NEUTRAL   30 non-null       float64
dtypes: float64(3), object(7)
memory usage: 81.6+ MB


### Paso 3: Carga del Modelo de Análisis de Sentimiento

En esta sección, cargamos el modelo de IA pre-entrenado (`distilroberta`) y su tokenizador. La selección de este modelo se basa en su especialización en el análisis de sentimiento de noticias financieras. También configuramos el entorno para que use la GPU (`cuda`) si está disponible, lo cual acelera drásticamente las predicciones.

### Paso 4: Definición de la Función de Predicción

Se crea una función `get_sentiment` que encapsula toda la lógica del análisis. Toma un texto de entrada, lo procesa para el modelo (tokenización y conversión a tensores), realiza la predicción de sentimiento y devuelve la etiqueta de sentimiento (Positivo, Negativo, Neutral) y sus probabilidades.

In [ ]:
# Cargar el modelo y el tokenizer
if df is not None:
    model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    print("\nModelo y tokenizer cargados correctamente.")

    # Definir la función de predicción
    def get_sentiment(text):
        if not isinstance(text, str) or pd.isna(text):
            return 'N/A', np.nan, np.nan, np.nan

        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        probabilities = torch.softmax(outputs.logits, dim=1)[0].cpu().numpy()

        label_mapping = {
            0: 'Negativo',
            1: 'Neutral',
            2: 'Positivo'
        }

        predicted_class_id = np.argmax(probabilities)
        sentiment = label_mapping[predicted_class_id]

        prob_negative = probabilities[0]
        prob_neutral = probabilities[1]
        prob_positive = probabilities[2]

        return sentiment, prob_positive, prob_negative, prob_neutral

    # Aplicar el análisis solo a los titulares en inglés
    print("\nIniciando el análisis de sentimiento para titulares en inglés ('en')...")

    # Obtener los índices de los registros en inglés
    english_indices = df[df['IDIOMA'] == 'en'].index

    # Iterar solo sobre los registros en inglés
    for index in english_indices:
        titular = df.loc[index, 'TITULAR']
        sentiment, prob_pos, prob_neg, prob_neut = get_sentiment(titular)

        # Llenar las columnas de sentimiento solo para los registros en inglés
        df.loc[index, 'SENTIMIENTO_RESULTADO'] = sentiment
        df.loc[index, 'PROBABILIDAD_POSITIVO'] = prob_pos
        df.loc[index, 'PROBABILIDAD_NEGATIVA'] = prob_neg
        df.loc[index, 'PROBABILIDAD_NEUTRAL'] = prob_neut

    print("Análisis de sentimiento completado.")
    print("\nDataFrame con los resultados finales:")
    print(df[['ID', 'FECHA', 'TITULAR', 'IDIOMA', 'SENTIMIENTO_RESULTADO', 'PROBABILIDAD_POSITIVO']].head())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]


Modelo y tokenizer cargados correctamente.

Iniciando el análisis de sentimiento para titulares en inglés ('en')...


Se aplican las predicciones del modelo a los datos del DataFrame. El código filtra los registros para analizar solo aquellos con el idioma 'en' (inglés). Luego, itera sobre cada titular, llama a la función de predicción y llena las columnas de sentimiento y probabilidad en el DataFrame.

In [ ]:
print(df[['FECHA', 'TITULAR', 'sentimiento_resultado', 'probabilidad_positivo', 'probabilidad_negativa', 'probabilidad_neutral']].head())

In [ ]:
# Guardar el resultado final en un nuevo archivo CSV en Drive
output_file_path = 'analisis_en.csv'
df.to_csv(output_file_path, index=False)
print(f"\nResultados guardados en '{output_file_path}'.")

In [ ]:
df.info()

Con esta muestra y conversión pudimos comprobar el funcionamiento del modelo con el formato que tendrán las noticias por analizar para nuestro flujo. En la base de datos de Snowflake tendremos noticias con los campos vacíos que deberán llenarse al ser consultados y procesados por el N8N. Luego regresarían para rellenar las tabla del SnowFlake.

# TFM GRUPO 4